In [13]:
import numpy as np

class FuzzyACController:
    def __init__(self):
        # Range untuk suhu (dalam Celsius)
        self.temp_range = np.arange(16, 35, 1)
        # Range untuk kelembaban (dalam %)
        self.humidity_range = np.arange(0, 101, 1)
        # Range untuk kecepatan kipas (dalam %)
        self.fan_speed_range = np.arange(0, 101, 1)

    def membership_temp(self, temp):
        """Fungsi keanggotaan untuk suhu"""
        temp_cold = np.zeros_like(self.temp_range, dtype=np.float64)
        temp_normal = np.zeros_like(self.temp_range, dtype=np.float64)
        temp_hot = np.zeros_like(self.temp_range, dtype=np.float64)

        # Dingin: 16-22°C
        for i, t in enumerate(self.temp_range):
            if t <= 18:
                temp_cold[i] = 1
            elif 18 < t <= 22:
                temp_cold[i] = (22 - t) / (22 - 18)
            else:
                temp_cold[i] = 0

        # Normal: 20-26°C
        for i, t in enumerate(self.temp_range):
            if 20 <= t <= 23:
                temp_normal[i] = (t - 20) / (23 - 20)
            elif 23 < t <= 26:
                temp_normal[i] = (26 - t) / (26 - 23)
            else:
                temp_normal[i] = 0

        # Panas: 24-35°C
        for i, t in enumerate(self.temp_range):
            if 24 <= t <= 28:
                temp_hot[i] = (t - 24) / (28 - 24)
            elif t > 28:
                temp_hot[i] = 1
            else:
                temp_hot[i] = 0

        idx = np.abs(self.temp_range - temp).argmin()
        return {
            'cold': temp_cold[idx],
            'normal': temp_normal[idx],
            'hot': temp_hot[idx]
        }

    def membership_humidity(self, humidity):
        """Fungsi keanggotaan untuk kelembaban"""
        humid_dry = np.zeros_like(self.humidity_range, dtype=np.float64)
        humid_normal = np.zeros_like(self.humidity_range, dtype=np.float64)
        humid_wet = np.zeros_like(self.humidity_range, dtype=np.float64)

        # Kering: 0-40%
        for i, h in enumerate(self.humidity_range):
            if h <= 30:
                humid_dry[i] = 1
            elif 30 < h <= 40:
                humid_dry[i] = (40 - h) / (40 - 30)
            else:
                humid_dry[i] = 0

        # Normal: 35-65%
        for i, h in enumerate(self.humidity_range):
            if 35 <= h <= 50:
                humid_normal[i] = (h - 35) / (50 - 35)
            elif 50 < h <= 65:
                humid_normal[i] = (65 - h) / (65 - 50)
            else:
                humid_normal[i] = 0

        # Basah: 60-100%
        for i, h in enumerate(self.humidity_range):
            if 60 <= h <= 80:
                humid_wet[i] = (h - 60) / (80 - 60)
            elif h > 80:
                humid_wet[i] = 1
            else:
                humid_wet[i] = 0

        idx = np.abs(self.humidity_range - humidity).argmin()
        return {
            'dry': humid_dry[idx],
            'normal': humid_normal[idx],
            'wet': humid_wet[idx]
        }

    def fuzzy_rules(self, temp_memb, humid_memb):
        """Aturan fuzzy untuk menentukan kecepatan kipas"""
        rules = []
        
        # Rule 1: Jika suhu DINGIN dan kelembaban KERING, maka kecepatan LAMBAT
        rules.append(min(temp_memb['cold'], humid_memb['dry']))
        
        # Rule 2: Jika suhu DINGIN dan kelembaban NORMAL, maka kecepatan LAMBAT
        rules.append(min(temp_memb['cold'], humid_memb['normal']))
        
        # Rule 3: Jika suhu NORMAL dan kelembaban NORMAL, maka kecepatan SEDANG
        rules.append(min(temp_memb['normal'], humid_memb['normal']))
        
        # Rule 4: Jika suhu PANAS atau kelembaban BASAH, maka kecepatan CEPAT
        rules.append(max(temp_memb['hot'], humid_memb['wet']))
        
        # Output membership untuk setiap rule (dalam urutan: LAMBAT, LAMBAT, SEDANG, CEPAT)
        speed_memberships = [20, 20, 50, 100]
        
        if sum(rules) == 0:
            return 50  # Default speed jika tidak ada rule yang aktif
            
        return sum(r * s for r, s in zip(rules, speed_memberships)) / sum(rules)

    def get_fan_speed(self, temperature, humidity):
        """Fungsi utama untuk mendapatkan kecepatan kipas"""
        temp_membership = self.membership_temp(temperature)
        humid_membership = self.membership_humidity(humidity)
        fan_speed = self.fuzzy_rules(temp_membership, humid_membership)
        return round(fan_speed, 2)

def validate_input(prompt, min_val, max_val):
    """Fungsi untuk memvalidasi input dari user"""
    while True:
        try:
            value = float(input(prompt))
            if min_val <= value <= max_val:
                return value
            print(f"Nilai harus antara {min_val} dan {max_val}")
        except ValueError:
            print("Masukkan angka yang valid!")

def main():
    controller = FuzzyACController()
    
    print("\n=== Sistem Kontrol AC Fuzzy Logic ===")
    print("====================================")
    
    while True:
        print("\nMasukkan nilai suhu dan kelembaban:")
        # Input suhu dengan validasi
        temp = validate_input("Suhu ruangan (16-35°C): ", 16, 35)
        
        # Input kelembaban dengan validasi
        humid = validate_input("Kelembaban ruangan (0-100%): ", 0, 100)
        
        # Hitung kecepatan kipas
        speed = controller.get_fan_speed(temp, humid)
        
        print("\n=== Hasil Analisis ===")
        print(f"Suhu: {temp}°C")
        print(f"Kelembaban: {humid}%")
        print(f"Kecepatan Kipas: {speed}%")
        
        # Interpretasi hasil
        if speed < 30:
            status = "LAMBAT"
        elif speed < 70:
            status = "SEDANG"
        else:
            status = "CEPAT"
        print(f"Status Kipas: {status}")
        

        lanjut = input("\nIngin memasukkan nilai lagi? (y/n): ").lower()
        if lanjut != 'y':
            break
    
    print("\nTerima kasih telah menggunakan sistem kontrol AC Fuzzy Logic!")

if __name__ == "__main__":
    main()


=== Sistem Kontrol AC Fuzzy Logic ===

Masukkan nilai suhu dan kelembaban:


Suhu ruangan (16-35°C):  29
Kelembaban ruangan (0-100%):  30



=== Hasil Analisis ===
Suhu: 29.0°C
Kelembaban: 30.0%
Kecepatan Kipas: 100.0%
Status Kipas: CEPAT



Ingin memasukkan nilai lagi? (y/n):  N



Terima kasih telah menggunakan sistem kontrol AC Fuzzy Logic!
